<a href="https://colab.research.google.com/github/spirosChv/smartNetsWorkshop/blob/main/neuron/practical2_basicNeuron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The NEURON Simulator

In [ ]:
# @title Make nicer plots -- Execute this cell
import matplotlib.pyplot as plt


def mystyle():
  """
  Create custom plotting style.

  Returns
  -------
  my_style : dict
      Dictionary with matplotlib parameters.

  """
  # color pallette
  style = {
      # Use LaTeX to write all text
      "text.usetex": False,
      "font.family": "DejaVu Sans",
      "font.weight": "bold",
      # Use 16pt font in plots, to match 16pt font in document
      "axes.labelsize": 16,
      "axes.titlesize": 20,
      "font.size": 16,
      # Make the legend/label fonts a little smaller
      "legend.fontsize": 14,
      "xtick.labelsize": 14,
      "ytick.labelsize": 14,
      "axes.linewidth": 2.5,
      "lines.markersize": 10.0,
      "lines.linewidth": 2.5,
      "xtick.major.width": 2.2,
      "ytick.major.width": 2.2,
      "axes.labelweight": "bold",
      "axes.spines.right": False,
      "axes.spines.top": False
  }

  return style


plt.style.use("seaborn-colorblind")
plt.rcParams.update(mystyle())

# What is NEURON?

The NEURON simulation environment is a powerful engine for performing simulations of neurons and biophysical neural networks. It permits the construction of biologically realistic membranes with active and passive ion channels, combined with virtual connectivity and electrophysiology tools to drive and measure neuron and network behaviors.

In [ ]:
!pip install neuron --quiet

In [ ]:
# @title Create the mod file `vecstim.mod`
# @markdown Execute this cell.
with open("vecstim.mod", "w") as file:
  file.write("""
    : Vector stream of events

    NEURON {
      THREADSAFE
      ARTIFICIAL_CELL VecStim
      RANGE delay
    }

    ASSIGNED {
      index
      etime (ms)
      space
      delay (ms)
    }

    PARAMETER { 
      ::delay = 0.0
    } 

    INITIAL {
      index = 0
      : delay = 0
      element()
      if (index > 0) {
        net_send(delay + etime - t, 1)
      }
    }

    NET_RECEIVE (w) {
      if (flag == 1) {
        net_event(t)
        element()
        if (index > 0) {
          net_send(delay + etime - t, 1)
        }
      }
    }

    VERBATIM
    extern double* vector_vec();
    extern int vector_capacity();
    extern void* vector_arg();
    ENDVERBATIM

    PROCEDURE element() {
    VERBATIM	
    { void* vv; int i, size; double* px;
      i = (int)index;
      if (i >= 0) {
        vv = *((void**)(&space));
        if (vv) {
          size = vector_capacity(vv);
          px = vector_vec(vv);
          if (i < size) {
            etime = px[i];
            index += 1.;
          }else{
            index = -1.;
          }
        }else{
          index = -1.;
        }
      }
    }
    ENDVERBATIM
    }

    PROCEDURE play() {
    VERBATIM
      void** vv;
      vv = (void**)(&space);
      *vv = (void*)0;
      if (ifarg(1)) {
        *vv = vector_arg(1);
      }
    ENDVERBATIM
    }
  """)

In [ ]:
# Remove old executables and compile with `nrnivmodl`
!rm -rf x86_64/  # UINIX specific command
!nrnivmodl  # UINIX specific command

## Step 1: Import the neuron module into Python.

We begin by loading NEURON’s h module.

In [ ]:
import neuron
print(neuron.__version__)

There are only a limited number of functions avaiable directly from the neuron module. In practice, we usually want to directly import the submodules we need; i.e., do something like the below instead:

In [ ]:
from neuron import h

When using NEURON, you will always want the h submodule. You may or may not need to import the additional submodules mentioned above. If you do, they can be imported separately or loaded in one line with a comma separated list, as in:

In [ ]:
from neuron import h, rxd

NEURON assumes certain default units (concentration in mM, time in ms, voltage in mV), but units can be specified explicitly by importing unit definitions from neuron.units. Even if you are using the default units, being explicit makes your code more readable by others. For example:

In [ ]:
from neuron.units import ms, mV

If the above gives you an error, then you are using a version of NEURON older than 7.7. Update before proceeding.

## Step 2: Create a cell

A `Section` is the basic morphological building-block in NEURON. We typically think of a Section as an unbranched cable, but it can also be used to represent a soma. Thus a simple model neuron with only a soma can be created as in:

In [ ]:
soma = h.Section(name='soma')

Aside 1: NEURON's `h.topology` function.

NEURON's `h.topology()` function displays the topological structure of the entire model, indicating which sections are connected to which sections, where they are connected, and how many segments each section is divided into.

If you're following along with our example, there's not much to see yet since there is only one section, but it does demonstrate that the soma has been created and has one segment (one dash is shown):

In [ ]:
h.topology()

The `h.topology()` function displays its data to screen and returns 1.0 indicating success (this function always succeeds). Note: This function is only for displaying data; other methods must be used to store the data in a variable for programmatic analysis.

### Aside 2: NEURON’s psection() function.

Every NEURON section has a `psection` method (think: properties of the section) that returns a Python dictionary providing a structured data representation of the properties of the section.

For example, we can query the soma via:

In [ ]:
soma.psection()

The results tell us the soma is a cylinder with length $100 microns$, diameter $500 microns$, axial resistivity $35.4 ohm*cm$, and specific membrance capacitance $1 \mu F/cm^2$

**Note:** calling this method does not itself print anything to the screen. Instead it returns a dictionary. We see the contents of the dictionary only because we are running interactively; from a script, nothing would be printed unless we explicitly printed it with print, or, better pretty-printed it with pprint.pprint.

Since this is a dictionary, we can extract any properties we want using square brackets. For example, the length of the section is:

In [ ]:
soma.psection()['morphology']['L']

All of these values can be individually accessed in more efficient ways, but psection provides an overview of the full properties of the section.

For example, the length of the soma is more efficiently available (and settable) via:

In [ ]:
soma.L

`soma.psection()['morphology']['diam']` is a list (of length 1 here), with each entry corresponding to the value for each segment. Similarly for `soma.psection()['cm']`, etc.

Important: You may have noticed that the default diameter is $500 \mu m$, which is excessively large for mammalian neurons. It's the default because it's appropriate for the squid giant axons studied by Hodgkin and Huxley. NEURON also uses squid-relevant values for axial resistivity (soma.Ra) and temperature (`h.celsius`). These should all be adjusted for mammalian models.

### Aside 3: Python’s dir() function.

We can also probe objects with Python’s built-in `dir()` function. Let’s see what it says about soma.

In [ ]:
dir(soma)

This tells us all of the Python methods and variables associated with the object. Any methods with two leading and trailing underscores are reserved by Python and may or may not be implemented by the object. The other items in the list are additional members of soma that we can call. To see all of the functions available to the neuron variable h, try calling `dir(h)`.

### Aside 4: Getting more help.

In addition to probing objects with `dir()`, help from docstrings is available using `help()`.

```python
help(object_or_variable)
```

For example, from `dir(soma)`, we know that `connect` is available. Let’s inquire about that.

In [ ]:
help(soma.connect)

## Step 3: Set the cell's morphological properties

Since we're simulating a soma, the default length of $100 \mu m$ and diameter of $500 \mu m$ are inappropriate. Let's set the length (`L`) and diameter (`diam`) to $20 \mu m$ instead:

In [ ]:
soma.L = 20
soma.diam = 20

In many models, you will have cells consisting of many connected sections. In brief, this can be done using the Section's connect method. That will be described in a subsequent part of the tutorial. For now though, we consider only the soma.

## Step 4: Biophysical Mechanisms

NEURON comes with a few built in biophysical mechanisms that can be added to a model.

- `pas`: Passive (“leak”) channel.
- `extracellular`: For simulating effects of nonzero extracellular potential, as may happen with leaky patch clamps, or detailed propertes of the myelin sheath.
- `hh`:	Hodgkin-Huxley sodium, potassium, and leakage channels.

### Aside 3: Insert a passive mechanism.

We see from the list of elements after calling `dir(soma)` that `insert` is available. This is the method we will use to insert mechanisms into the membrane. Let’s insert a passive leak conductance across the membrane and do this by passing `pas` as the mechanism type:

In [ ]:
soma.insert('pas')

### IMPORTANT: Sections and segments.

A NEURON Section is considered a piece of cylindrical cable. Depending on the resolution desired, it may be necessary to divide the cable into a number of segments where voltage varies linearly between centers of adjacent segments. The number of segments within a section is given by the variable, `nseg`. The total ionic current across the segment membrane is approximately the area of the segment multiplied by the ionic current density at the center of the segment. To access a part of the section, specify a value between 0 and 1, where 0 is typically the end closest to the soma and 1 is the distal end. Because nseg divides the cable into equal-length parts, it should be an odd number so that to address the middle of the cable, (0.5), gives the middle segment.

To summarize, we access sections by their name and segments by some location on the section.

Section: `section`
Segment: `section(loc)`


Using the Python `type()` function can tell us what a variable is:

In [ ]:
print("type(soma) = {}".format(type(soma)))
print("type(soma(0.5)) ={}".format(type(soma(0.5))))

### Aside 4: Accessing segment variables.

Segment variables follow the idiom:

```python
section(loc).var
```

And for mechanisms on the segment:

```python
section(loc).mech.var
```

or

```python
section(loc).var_mech
```

The first form is preferred.

In [ ]:
mech = soma(0.5).pas
print(dir(mech))

In [ ]:
print(mech.g)
print(soma(0.5).pas.g)

## Step 5a: Insert an alpha synapse.

Let’s insert an AlphaSynapse object onto the soma to induce some membrane dynamics.

In [ ]:
asyn = h.AlphaSynapse(soma(0.5))

AlphaSynapse is a **Point Process**. Point processes are point sources of current. When making a new Point Process, you pass the segment to which it will bind.

Again, with `dir()` function, we can validate that asyn is an object and contains some useful parameters. Let’s look at some of those parameters.

In [ ]:
dir(asyn)

In [ ]:
print("asyn.e = {}".format(asyn.e))
print("asyn.gmax = {}".format(asyn.gmax))
print("asyn.onset = {}".format(asyn.onset))
print("asyn.tau = {}".format(asyn.tau))

Let’s assign the onset of this synapse to occur at 20 ms and the maximal conductance to 1.

In [ ]:
asyn.onset = 20
asyn.gmax = 1

Let’s look at the state of our cell using neuron’s `psection()`.

In [ ]:
h.psection()

# Step 5b: Insert a stimulus

Let's insert a current clamp (an `IClamp` object) into the center of the `soma` to induce some membrane dynamics.

In [ ]:
iclamp = h.IClamp(soma(0.5))

An `IClamp` is a **Point Process**. Point processes are point sources of current. When making a new Point Process, you pass the segment to which it will bind.

Again, with the `dir` function, we can validate that `iclamp` is an object and contains some useful parameters. Let's look at some of those parameters. We use a [list comprehension](https://www.pythonforbeginners.com/basics/list-comprehensions-in-python) to ignore those elements of the dir that start with double underscores (and are thus Python magic methods and not functions/variables intended to be used directly).

In [ ]:
print([item for item in dir(iclamp) if not item.startswith('__')])

In particular, we notice three key properties of a current clamp: amp -- the amplitude (in nA), delay -- the time the current clamp switches on (in ms), and dur -- how long (in ms) the current clamp stays on. Let's set these values:

In [ ]:
iclamp.delay = 2
iclamp.dur = 0.1
iclamp.amp = 0.9

Let's use psection to get a representation of the soma model:

In [ ]:
soma.psection()

## Step 6: Set up recording variables.

The cell should be configured to run a simulation. However, we need to set up variables we wish to record from the simulation. For now, we will record the membrane potential, which is `soma(0.5).v`. References to variables are available as `_ref_rangevariable`.

In [ ]:
v_vec = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

## Step 6: Run the simulation.

To run the simulation, we execute the following lines.

In [ ]:
h.load_file('stdrun.hoc')

```python
h.tstop = 40.0
h.run()
```

**Note:** If we had not included gui in the list of things to import, we would have also had to execute:

`h.load_file('stdrun.hoc')`

which defines the `run()` function (the alternative would be to specify initialization and advance in more detail).

Nowadays, NEURON guidelines use only the `continuerun` function.

In [ ]:
def myinit(temp=34, dt=0.1, cvode=True):
    """
    Initialize function.

    Parameters
    ----------
    temp: float, optional
        Tmperature in degC. The default is 34.
    dt : float, optional
        Integration time-step in ms. The default is 0.1.
    cvode: bool, optional
        Enable CVODE for faster simulations. The default is `True`.

    Returns
    -------
    None.

    """
    # Enable CVODE for faster Simulations
    h.CVode().active(cvode)
    # new code to happen after initialization here
    print('Initializing NEURON...')
    # only need the following if states have been changed
    h.celsius = temp
    h.stdinit()
    h.init()
    h.finitialize()
    if h.cvode.active():
        h.cvode.re_init()
    else:
        h.fcurrent()
    h.frecord_init()
    print(f'Initialization has been completed. Temperature is: {temp} degC')

In [ ]:
myinit()
h.continuerun(1000 * ms)

## Step 7: Plot the results.

We utilize the pyplot module from the matplotlib Python package to visualize the output.

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 6))
plt.plot(t_vec, v_vec)
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()

The last line displays the graph and allows you to interact with it (zoom, pan, save, etc). NEURON and Python will wait until you close the figure window to proceed.

## Step 8: Saving and restoring results.

NEURON Vector objects can be pickled. This allows them to be stored to disk and restored later:

Saving:

In [ ]:
# Pickle
import pickle
with open('t_vec.p', 'wb') as t_vec_file:
  pickle.dump(t_vec, t_vec_file)
with open('v_vec.p', 'wb') as v_vec_file:
  pickle.dump(v_vec, v_vec_file)

In [ ]:
# Unpickle
with open('t_vec.p', 'rb') as t_vec_file:
  t_vec_new = pickle.load(t_vec_file)
with open('v_vec.p', 'rb') as vec_file:
  v_vec_new = pickle.load(vec_file)

# Confirm
plt.figure(figsize=(10, 6))
plt.plot(t_vec_new, v_vec_new)
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.show()

## Extra Step: Generate a train of presynaptic stimuli

A `NetStim` is a spike generator that can be used as the source in a `NetCon`, behaving as external input onto the synapse of a target cell. The following code makes a `NetStim` object that generates one spike at time $t=9$. The NetCon then adds another ms delay to deliver a synaptic event at time $t=10$ onto the first cell.

The code below makes a stimulator and attaches it to a synapse object ([ExpSyn](https://neuron.yale.edu/neuron/static/py_doc/modelspec/programmatic/mechanisms/mech.html#ExpSyn)) that behaves much like an AMPA synapse – it conducts current as a decaying exponential function.

In [ ]:
stim = h.NetStim()  # Make a new stimulator

# Attach it to a synapse in the middle of the dendrite
# of the first cell in the network. (Named 'syn_' to avoid
# being overwritten with the 'syn' var assigned later.)
syn_ = h.ExpSyn(soma(0.5))

stim.number = 1
stim.start = 9
ncstim = h.NetCon(stim, syn_)
ncstim.delay = 1
ncstim.weight[0] = 0.04  # NetCon weight is a vector.

Exponentially decaying currents, such as that generated by the synapse `syn_` have dynamics that depend on tau, the time constant. Let's specify a time constant of $2 ms$:

In [ ]:
syn_.tau = 2

The parameter `tau` specifies how quickly the currents decay. The exact value of the current depends on the cell's membrane potential, and the synapse's reversal potential, `syn_.e`.

In [ ]:
print(f'Reversal potential = {syn_.e} mV')

NetCon syntax

```python
netcon = h.NetCon(source_ref_v, target, [threshold, delay, weight], sec=section)

netcon = h.NetCon(source, target, [threshold, delay, weight])
```

**Note:** The threshold is used when two NEURON sections are connected via a Synapse and defines the presynaptic spike.

Example:

When you connect two neurons:
```python
nc = h.NetCon(soma(0.5)._ref_v, syn, sec=soma)
nc.threshold = 10  # mV
```

When you connect an artificial neuron with a neuron:
```python
nc = h.NetCon(inputCell, syn)
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def poisson_spikes(t1, t2, N, rate=10, dt=0.1):
    """
    Poisson spike generator.
    Parameters
    ----------
    t1 : float
        Simulation time start in miliseconds (ms).
    t2 : float
        Simulation time end in miliseconds (ms).
    N : int, optional
        Number of presynaptic spikes.
    rate : float, optional
        Mean firing rate in Hz. The default is 10.
    dt : float, optional
        Time step in ms. The default is 0.1.
    Returns
    -------
    spks : TYPE
        DESCRIPTION.
    """
    spks = []
    tarray = np.arange(t1, t2+dt, dt)
    for n in range(N):
        spkt = tarray[np.random.rand(len(tarray)) < rate*dt/1000.]  # Determine list of times of spikes
        idx = [n]*len(spkt)  # Create vector for neuron ID number the same length as time
        spkn = np.concatenate([[idx], [spkt]], axis=0).T  # Combine tw lists
        if len(spkn) > 0:
            spks.append(spkn)
    spks = np.concatenate(spks, axis=0)
    return spks


N = 50  # number of presynaptic neurons
spks = poisson_spikes(t1=0, t2=1000, N=N, rate=15, dt=0.1)

plt.figure(figsize=(10, 6))
for i in range(N):
  plt.scatter(spks[spks[:, 0]==i][:,1],
              i*np.ones(len(spks[spks[:, 0]==i][:,1])),
              color='k')
plt.xlabel('time (ms)')
plt.ylabel('neuron id')
plt.show()

### How we load external inputs on NEURON with `VecStim`.

In [ ]:
spiketimes = spks[spks[:, 0]==i][:,1]
vecSpikes = h.Vector(spiketimes)
vecStim = h.VecStim()
vecStim.play(vecSpikes)

### Create a new `Section`.

In [ ]:
soma = h.Section(name='soma')
soma.L = 10
soma.diam = 10
soma.nseg = 11
soma.Ra = 250
soma.cm = 1

soma.insert('hh')
for seg in soma:
  seg.hh.gnabar = 0.12  # Sodium conductance in S/cm2
  seg.hh.gkbar = 0.036  # Potassium conductance in S/cm2
  seg.hh.gl = 0.0003    # Leak conductance in S/cm2
  seg.hh.el = -54.3     # Reversal potential in mV

dend = h.Section(name='dend')
dend.L = 500
dend.diam = 3
dend.nseg = 11
dend.Ra = 250
dend.cm = 1.4

# Insert passive current in the dendrite
dend.insert('pas')                 
for seg in dend:
  seg.pas.g = 0.001  # Passive conductance in S/cm2
  seg.pas.e = -65    # Leak reversal potential mV

### Let's connect our compartments!

```python
child.connect(parent, [0 or 1])
child.connect(parent(x), [0 or 1])
```

The first form connects the child at end 0 or 1 to the parent section at position x. By default the child end 0 connects to the parent end 1. An alternative syntax is the second form in which the location on the parent section is indicated. If a section is connected twice a Notice is printed on the standard error device saying that the section has been reconnected (the last connection takes precedence). To avoid the notice, disconnect the section first with the function `disconnect()`. If sections are inadvertently connected in a loop, an error will be generated when the internal data structures are created and the user will be required to disconnect one of the sections forming the loop.

```python
soma = h.Section(name='soma')
axon = h.Section(name='axon')
axon.connect(soma(0), 0)
```

This means that we connect the Section `axon` @ 0 with `soma` @ 0.

In [ ]:
dend.connect(soma(0.5), 0)

### Create a synapse

In [ ]:
syn_2 = h.ExpSyn(dend(1))
syn_2.tau = 20

ncstim2 = h.NetCon(vecStim, syn_2)
ncstim2.delay = 1
ncstim2.weight[0] = 0.01  # NetCon weight is a vector.

In [ ]:
v_vec_d = h.Vector().record(dend(0.5)._ref_v)  # Membrane potential vector
v_vec_s = h.Vector().record(soma(0.5)._ref_v)  # Membrane potential vector
t_vec = h.Vector().record(h._ref_t)  # Time stamp vector

myinit()
h.continuerun(1000 * ms)

plt.figure(figsize=(10, 6))
plt.plot(t_vec, v_vec_s, label='soma')
plt.plot(t_vec, v_vec_d, label='dend')
plt.xlabel('time (ms)')
plt.ylabel('mV')
plt.legend()
plt.show()